In [1]:
import pandas as pd
from geopy.distance import distance
from tqdm import tqdm

In [2]:
YONGE_FLOW_COLLECTION_POINT = (43.646882, -79.376952)
count_df = pd.read_csv("../../raw_data/traffic_volume/traffic_volume_raw-data-2020-2029.zip")

In [3]:
count_df

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,ex_peds,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other
0,1,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2,13462138,NaN,...,60,0,4,0,0,0,0,0,0,0
1,2,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2,13462138,NaN,...,54,0,3,0,0,0,0,0,0,0
2,3,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2,13462138,NaN,...,86,0,2,0,0,0,0,0,0,0
3,4,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2,13462138,NaN,...,105,0,2,1,0,0,0,0,0,0
4,5,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2,13462138,NaN,...,104,0,1,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123070,123071,47365,2022-05-25,5116,DONLANDS AVE AT OCONNOR DR (PX 443),-79.343116,43.694722,2,13458943,443.0,...,7,9,6,0,0,0,0,0,0,0
123071,123072,47365,2022-05-25,5116,DONLANDS AVE AT OCONNOR DR (PX 443),-79.343116,43.694722,2,13458943,443.0,...,10,9,2,0,0,1,0,0,0,0
123072,123073,47365,2022-05-25,5116,DONLANDS AVE AT OCONNOR DR (PX 443),-79.343116,43.694722,2,13458943,443.0,...,8,9,4,3,0,1,0,0,0,0
123073,123074,47365,2022-05-25,5116,DONLANDS AVE AT OCONNOR DR (PX 443),-79.343116,43.694722,2,13458943,443.0,...,4,7,2,1,0,2,0,0,0,0


In [4]:
count_df.columns

Index(['_id', 'count_id', 'count_date', 'location_id', 'location', 'lng',
       'lat', 'centreline_type', 'centreline_id', 'px', 'time_start',
       'time_end', 'sb_cars_r', 'sb_cars_t', 'sb_cars_l', 'nb_cars_r',
       'nb_cars_t', 'nb_cars_l', 'wb_cars_r', 'wb_cars_t', 'wb_cars_l',
       'eb_cars_r', 'eb_cars_t', 'eb_cars_l', 'sb_truck_r', 'sb_truck_t',
       'sb_truck_l', 'nb_truck_r', 'nb_truck_t', 'nb_truck_l', 'wb_truck_r',
       'wb_truck_t', 'wb_truck_l', 'eb_truck_r', 'eb_truck_t', 'eb_truck_l',
       'sb_bus_r', 'sb_bus_t', 'sb_bus_l', 'nb_bus_r', 'nb_bus_t', 'nb_bus_l',
       'wb_bus_r', 'wb_bus_t', 'wb_bus_l', 'eb_bus_r', 'eb_bus_t', 'eb_bus_l',
       'nx_peds', 'sx_peds', 'ex_peds', 'wx_peds', 'nx_bike', 'sx_bike',
       'ex_bike', 'wx_bike', 'nx_other', 'sx_other', 'ex_other', 'wx_other'],
      dtype='object')

In [3]:
count_df["time_start"] = pd.to_datetime(count_df["time_start"], utc=True)
count_df["time_start"] = count_df["time_start"].apply(lambda x: x.to_datetime64())
count_df["time_end"] = pd.to_datetime(count_df["time_end"], utc=True)
count_df["time_end"] = count_df["time_end"].apply(lambda x: x.to_datetime64())

In [4]:
count_df = count_df.drop(
    columns=["count_id", "count_date", "location_id", "centreline_type", "centreline_id", "px"])
count_df = count_df.drop(columns=list(
    filter(lambda x: "peds" in x, count_df.columns)))
count_df = count_df.drop(columns=list(
    filter(lambda x: "bike" in x, count_df.columns)))
count_df = count_df.drop(columns=list(
    filter(lambda x: "other" in x, count_df.columns)))

In [5]:
count_car_columns = list(filter(lambda x: "car" in x, count_df.columns))
count_truck_columns = list(filter(lambda x: "truck" in x, count_df.columns))
count_bus_columns = list(filter(lambda x: "bus" in x, count_df.columns))

In [6]:
count_df["car_count"] = count_df[count_car_columns].sum(axis=1)
count_df["truck_count"] = count_df[count_truck_columns].sum(axis=1)
count_df["bus_count"] = count_df[count_bus_columns].sum(axis=1)

In [7]:
count_df = count_df.drop(columns=count_car_columns)
count_df = count_df.drop(columns=count_truck_columns)
count_df = count_df.drop(columns=count_bus_columns)

In [8]:
count_cols = list(filter(lambda x: "count" in x, count_df.columns))
count_df["total_count"] = count_df[count_cols].sum(axis=1)
count_df = count_df.drop(columns=["car_count", "truck_count", "bus_count"])

In [9]:
tqdm.pandas()
count_df["distance_from_point"] = count_df.progress_apply(
    lambda x: distance((x["lat"], x["lng"]), YONGE_FLOW_COLLECTION_POINT).km, axis=1)

100%|██████████| 123075/123075 [00:20<00:00, 6070.62it/s]


In [10]:
count_df = count_df.drop(columns=["time_start"])

In [11]:
count_df["end_hour"] = count_df["time_end"].dt.hour
count_df["end_minute"] = count_df["time_end"].dt.minute
count_df["end_day_of_week"] = count_df["time_end"].dt.dayofweek

In [12]:
count_df.to_csv("../../csv/v2/030_yonge_count_processed.csv.zip", index=False, compression="zip")